In [1]:
import pandas as pd
import numpy as np

In [2]:
def read_values(name):
    return pd.read_csv(name, header=None, delim_whitespace=True).values;

def load_raw(group):
    raw = list()
    filenames = ['./%s/Inertial Signals/%s_%s_%s.txt' % (group,cmp,ax,group) 
                 for cmp in ['body_acc', 'body_gyro', 'total_acc'] for ax in ['x', 'y', 'z']]
    for n in filenames:
        raw.append(read_values(n))
    return np.dstack(raw)

In [3]:
trainX = load_raw('train')
trainy = read_values('./train/y_train.txt')
testX = load_raw('test')
testy = read_values('./test/y_test.txt')

In [4]:
from keras.utils import to_categorical
trainy = to_categorical(trainy-1)
testy = to_categorical(testy-1)
testX.shape, testy.shape, trainX.shape, trainy.shape

Using TensorFlow backend.


((2947, 128, 9), (2947, 6), (7352, 128, 9), (7352, 6))

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
def evaluate():
    verbose, epochs, batch_size = 0, 10, 32
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1] 
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=(n_timesteps,n_features))) 
    model.add(Conv1D(64, 3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax')) 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy


In [8]:
from datetime import datetime
results = list();
for r in range(10):
    start = datetime.now();
    ac = evaluate()
    du = datetime.now() - start;
    print('iteration: %s accuracy %f in %s' % (r,ac, du))
    results.append((ac,du))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
iteration: 0 accuracy 0.901256 in 0:01:04.253467
iteration: 1 accuracy 0.907024 in 0:01:13.519357
iteration: 2 accuracy 0.894130 in 0:01:06.056280
iteration: 3 accuracy 0.900577 in 0:01:05.024860
iteration: 4 accuracy 0.915507 in 0:01:07.736438
iteration: 5 accuracy 0.899220 in 0:01:06.181616
iteration: 6 accuracy 0.907363 in 0:01:07.211142
iteration: 7 accuracy 0.902952 in 0:01:04.673890
iteration: 8 accuracy 0.899559 in 0:01:06.134484
iteration: 9 accuracy 0.901934 in 0:01:07.213402


In [12]:
from numpy import mean
from numpy import std
[print('accuracy: %f , duration %s' % (p[0], p[1])) for p in results]
accuracies = [p[0] for p in results]
print('accuracy: mean %f, std %f' % (mean(accuracies), std(accuracies)))



accuracy: 0.901256 , duration 0:01:04.253467
accuracy: 0.907024 , duration 0:01:13.519357
accuracy: 0.894130 , duration 0:01:06.056280
accuracy: 0.900577 , duration 0:01:05.024860
accuracy: 0.915507 , duration 0:01:07.736438
accuracy: 0.899220 , duration 0:01:06.181616
accuracy: 0.907363 , duration 0:01:07.211142
accuracy: 0.902952 , duration 0:01:04.673890
accuracy: 0.899559 , duration 0:01:06.134484
accuracy: 0.901934 , duration 0:01:07.213402
accuracy: mean 0.902952, std 0.005536
